<a href="https://colab.research.google.com/github/vishal-verma27/Tkinter-NLTK-Chatbot/blob/main/Tkinter_NLTK_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Website: https://pykit.org                                                                                                                                      
#### Mail: info@pykit.org

---

#### About Pykit
Pykit is a free online analytics resource website. pykit was founded in 2021 by [Vishal Verma](https://www.linkedin.com/in/vishalverma2701/). The main goal of this site is to provide quality tools, tutorials, courses, and other online resources that allow anyone to master Data Research & Analytics based on real case studies and industry projects.

---
#### [GitHub Link](https://github.com/vishal-verma27/Tkinter-NLTK-Chatbot)
#### [Author Portfolio](https://eportfolio.mygreatlearning.com/vishal-verma2)

### Importing Libraries

In [ ]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

from keras.models import load_model

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("Train_Bot.json").read()
intents = json.loads(data_file)

### Data Pre-Processing

In [ ]:
# preprocessing the json data
# tokenization

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

128 documents
8 classes ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']
158 unique lemmatized words ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'classification', 'contact', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'explain', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'knn', 'later', 'layer', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'me', 'ml', 'my', 'naive', 'name', 'nb', 'net', 'network', 'neural', 'no', 'not', 'of', 'olympus', 'olypus', 'on', 'online', 'operation', 'opertions', 'otimi

### Creating Training Data

In [ ]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


### Creating NN Model

In [ ]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/100
26/26 [==============================] - 0s 838us/step - loss: 2.0777 - accuracy: 0.2031
Epoch 2/100
26/26 [==============================] - 0s 758us/step - loss: 2.0155 - accuracy: 0.2188
Epoch 3/100
26/26 [==============================] - 0s 798us/step - loss: 1.9006 - accuracy: 0.2109
Epoch 4/100
26/26 [==============================] - 0s 878us/step - loss: 1.8282 - accuracy: 0.2578
Epoch 5/100
26/26 [==============================] - 0s 718us/step - loss: 1.7235 - accuracy: 0.3828
Epoch 6/100
26/26 [==============================] - 0s 838us/step - loss: 1.6970 - accuracy: 0.3672
Epoch 7/100
26/26 [==============================] - 0s 838us/step - loss: 1.6093 - accuracy: 0.4297
Epoch 8/100
26/26 [==============================] - 0s 638us/step - loss: 1.4923 - accuracy: 0.4453
Epoch 9/100
26/26 [==============================] - 0s 718us/step - loss: 1.4023 - accuracy: 0.4766
Epoch 10/100
26/26 [==============================] - 0s 678us/step - loss: 1.2606 - accura

26/26 [==============================] - 0s 678us/step - loss: 0.0638 - accuracy: 0.9688
Epoch 82/100
26/26 [==============================] - 0s 678us/step - loss: 0.0218 - accuracy: 1.0000
Epoch 83/100
26/26 [==============================] - 0s 678us/step - loss: 0.0421 - accuracy: 1.0000
Epoch 84/100
26/26 [==============================] - 0s 678us/step - loss: 0.0246 - accuracy: 1.0000
Epoch 85/100
26/26 [==============================] - 0s 638us/step - loss: 0.0232 - accuracy: 1.0000
Epoch 86/100
26/26 [==============================] - 0s 678us/step - loss: 0.0756 - accuracy: 0.9766
Epoch 87/100
26/26 [==============================] - 0s 638us/step - loss: 0.0339 - accuracy: 0.9922
Epoch 88/100
26/26 [==============================] - 0s 678us/step - loss: 0.0258 - accuracy: 1.0000
Epoch 89/100
26/26 [==============================] - 0s 678us/step - loss: 0.0444 - accuracy: 0.9922
Epoch 90/100
26/26 [==============================] - 0s 678us/step - loss: 0.0310 - accuracy: 

### Repeating all the above steps and using the pickled model for the prediction of responses

In [ ]:
# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("Train_Bot.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("Bot: This is Sophie! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

## Chatting with BOT using the Command Line Option

In [ ]:
# start the chat bot
start_chat()

Bot: This is Sophie! Your Personal Assistant.


hey
Bot: Hello! how can i help you ?

--------------------------------------------------
hello
Bot: Hello! how can i help you ?

--------------------------------------------------
hi
Bot: Hello! how can i help you ?

--------------------------------------------------
there?
Bot: Hello! how can i help you ?

--------------------------------------------------
What is ML?
Bot: Link: Machine Learning wiki 

--------------------------------------------------
You are not a good assistant
Bot: Tarnsferring the request to your PM

--------------------------------------------------
end


## Chatting with BOT using the Tkinter App

#### **For running the Tkinter GUI you have to download this notebook in ipynb format and run using jupyter notebook in your local machine/pc because in google colab, you cannot run Tkinter apps.**

In [ ]:
import random
import tkinter as tk
from tkinter import *

root=tk.Tk()
filename="Chat Bot"
root.title(f"Chat Bot")
root.geometry('500x400')
root.resizable(False, False)
message=tk.StringVar()

chat_win=Frame(root,bd=1,bg='white',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=488)

textcon=tk.Text(chat_win,bd=1,bg='white',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=380)
mes_win.focus()

textcon.config(fg='black')
textcon.tag_config('usr',foreground='black')
textcon.insert(END,"Bot: This is Sophie! Your Personal Assistant.\n\n")
mssg=mes_win.get()

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

def send_msz(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'You: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='black')
        textcon.insert(END,"Bot: Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='black')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)

button_send=Button(root,text='Send',bg='dark green',activebackground='grey',command=send_msz,width=12,height=5,font=('Arial'))
button_send.place(x=376,y=310,height=60,width=110)
root.bind('<Return>', send_msz,button_send)
root.mainloop()

### END